In [10]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec

In [12]:
api_key = os.getenv("PINECONE_API_KEY")
print(f"API Key Loaded: {api_key}")

API Key Loaded: None
